In [49]:
import webbrowser
import requests
import msal

import os
os.chdir("/home/daniel/de_AI_Fabriek/belasting-agent-playground/")

from keys import O365_CLIENT_SECRET, O365_CLIENT_ID

SCOPES = ["User.Read"]

# AUTHORITY = 'https://login.microsoftonline.com/' + "b4790366-ebc1-458b-8baf-4aed4d12f0fc"
AUTHORITY = 'https://login.microsoftonline.com/common'

## msal

In [46]:
def generate_acces_token(app_id, scopes, authority):
    acces_token_cache = msal.SerializableTokenCache()
    
    # read token cache
    if os.path.exists("msg_api_token_access.json"):
        acces_token_cache.deserialize(open("msg_api_token_access.json", "r").read())
        
    client = msal.PublicClientApplication(
        app_id,
        authority=authority,
        )
    
    accounts = client.get_accounts()
    if accounts:
        token_response = client.acquire_token_silent(scopes, account=accounts[0])
    else:
        flow = client.initiate_device_flow(scopes=scopes)
        print('user code: ', flow['user_code'])
        webbrowser.open(flow['verification_uri'])
        token_response = client.acquire_token_by_device_flow(flow)
        print(token_response)
        
    with open("msg_api_token_access.json", "w") as _f:
        _f.write(acces_token_cache.serialize())
    return token_response
        
    

In [50]:
token_response = generate_acces_token(O365_CLIENT_ID, SCOPES, AUTHORITY)

user code:  D56Q45326
{'token_type': 'Bearer', 'scope': 'openid profile User.Read email', 'expires_in': 4132, 'ext_expires_in': 4132, 'access_token': 'eyJ0eXAiOiJKV1QiLCJub25jZSI6IlNRU3NFcjJnTEtiRXhyb2ZiYm5oTDNsdDRRNWptYXFwZm1BdUdzVUNKQVUiLCJhbGciOiJSUzI1NiIsIng1dCI6IllUY2VPNUlKeXlxUjZqekRTNWlBYnBlNDJKdyIsImtpZCI6IllUY2VPNUlKeXlxUjZqekRTNWlBYnBlNDJKdyJ9.eyJhdWQiOiIwMDAwMDAwMy0wMDAwLTAwMDAtYzAwMC0wMDAwMDAwMDAwMDAiLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8wMDMwZTJmOC05Njg4LTQ5YzgtYWU0Zi1jZWY2N2E3ZWM4ZmYvIiwiaWF0IjoxNzM4NDQyMDk0LCJuYmYiOjE3Mzg0NDIwOTQsImV4cCI6MTczODQ0NjUyNywiYWNjdCI6MCwiYWNyIjoiMSIsImFpbyI6IkFUUUF5LzhaQUFBQWIvaTVNdHVzQnFPZmpnQ0RhRWpKVDVLcTAwdTZSUWVzeEdIZCs4citSZjNvZm10OWNMUXZWckxpcCs1RmplNUgiLCJhbXIiOlsicHdkIl0sImFwcF9kaXNwbGF5bmFtZSI6ImJlbGFzdGluZy1hYW5naWZ0ZW4tYWdlbnQiLCJhcHBpZCI6IjZmZTRjOTlhLTc4ODYtNDQ2My05MWJiLWNmMTMzYzVkZDE4NCIsImFwcGlkYWNyIjoiMCIsImZhbWlseV9uYW1lIjoidmFuIE9vc3Rlcm9vbSIsImdpdmVuX25hbWUiOiJEYW5pZWwiLCJpZHR5cCI6InVzZXIiLCJpcGFkZHIiOiIyYTAyOmE0NDA6Nzkx